<div style="text-align:center">
    <h1> Datatypes </h1>
    <h2> CS3100 Monsoon 2020 </h2>
</div>

## Review

Previously

* Function definition and application
* Anonymous and recursive functions
* Tail call optimisation

This lecture,

* Data types

## Type aliases

OCaml supports the definition of aliases for existing types. For example,

In [ ]:
type int_float_pair = int * float

In [ ]:
let x = (10, 3.14)

In [ ]:
let y : int_float_pair = x

## Records

* Records in OCaml represent a collection of named elements. 
* A simple example is a point record containing x, y and z fields:

In [ ]:
type point = {
  x : int;
  y : int;
  z : int;
}

## Records: Creation and access

We can create instances of our point type using `{ ... }`, and access the elements of a point using the `.` operator:

In [ ]:
let origin = { y = 0; x = 0;z = 0 }

let get_y r = r.y

## Records: Functional update

* New records can also be created from existing records using the `with` keyword.

In [ ]:
let p = { origin with z = 10 }

* `p` is a new record with the same fields as `origin` except `z`.
* `origin` remains unchanged!

In [ ]:
origin

## Records: Field punning

Another useful trick with records is field punning, which allows you to replace:

In [ ]:
let mk_point x y z = { x = x; y = y; z = z }

with 

In [ ]:
let mk_point x y z = { x; y; z }

## Product Types

* Records and tuples are known as **product types**. 
  + Each value of a product type includes all of the values that constitute the product. 
  
```ocaml
type person_r = {name: string; age: int; height: float}
type person_t = string * int * float
```

* Records are indexed by *names* whereas *tuples* are indexed by positions (1st, 2nd, etc.).

## Is there a _sum_ type?

<center>

<h1> VARIANTS </h1>
</center>

## Defining variants

The type definition syntax is:

```ocaml
type t = 
| C1
| C2 of t2
| C3 of t3
| ...
```

* C1, C2, C2 are known as constructors
* t2 and t3 are types of _optional_ data carried by constructor
* Also known as **Algebraic Data Types**

In [1]:
type color = 
  | Red
  | Green
  | Blue

type color = Red | Green | Blue


In [2]:
let v = (Green , Red)

val v : color * color = (Green, Red)


In [3]:
type point = {x : int; y : int}

type shape = 
  | Circle     of point * float (* center, radius *)
  | Rectangle  of point * point (* lower-left, upper-right *)
  | ColorPoint of point * color

type point = { x : int; y : int; }


type shape =
    Circle of point * float
  | Rectangle of point * point
  | ColorPoint of point * color


In [4]:
Circle ({x=4;y=3}, 2.5)

- : shape = Circle ({x = 4; y = 3}, 2.5)


In [6]:
Rectangle ({x=3;y=4}, {x=7;y=9})

- : shape = Rectangle ({x = 3; y = 4}, {x = 7; y = 9})


## Question

In `Rectangle ({x=3;y=4}, {x=7;y=9})`, which point is the lower-right?

**None!**

`Rectangle ({x=3;y=4} (* lower-left *), 
           {x=7;y=9} (* upper-right *))`

## Prefer record-types over tuples

In [11]:
type shape = 
  | Circle     of {center: point; radius: float}
  | Rectangle  of {lower_left: point; upper_right: point}
  | ColorPoint of point * color

type shape =
    Circle of { center : point; radius : float; }
  | Rectangle of { lower_left : point; upper_right : point; }
  | ColorPoint of point * color


In [12]:
Rectangle ({x=3;y=4}, {x=7;y=9})

error: compile_error

In [13]:
Rectangle {lower_left={x=3;y=4}; upper_right={x=7;y=9}}

- : shape =
Rectangle {lower_left = {x = 3; y = 4}; upper_right = {x = 7; y = 9}}


## Recursive variant types

Let's define an integer list

In [ ]:
type intlist = 
  | INil
  | ICons of int * intlist

In [ ]:
ICons (1, ICons (2, ICons (3, INil)))

* `Nil` and `Cons` originate from Lisp.

## String List

```ocaml
type stringlist =
  | SNil
  | Scons of string * stringlist
```

* Now what about `pointlist`, `shapelist`, etc?

## Parameterized Variants

In [ ]:
type 'a lst = 
    Nil
  | Cons of 'a * 'a lst

In [ ]:
Cons (1, Cons (2, Nil))

In [ ]:
Cons ("Hello", Cons("World", Nil))

## 'a is a Type Variable

`type 'a list = ...`

* **Variable**: name standing for an unknown value
* **Type Variable**: name standing for an unknown type

* Java example is `List<T>`
* C++ example is `std::vector<T>`

* OCaml syntax for type variable is a single quote followed by an identifier
  + '`foo`, `'key`, '`value`
* Most often just `'a`, `'b`. 
  + Pronounced "alpha", "beta" or "quote a", "quote b".

## Polymorphism

* The type `'a lst` that we had defined earlier is a **polymorphic data type**.
  + poly = many, morph = change. 
* Write functionality that works for many data types.
  + Related to Java Generics and C++ template instantiation.
* In `'a lst`, `lst` is known as a **type constructor**.
  + constructs types such as `int lst`, `string lst`, `shape lst`, etc.

## OCaml built-in lists are just variants

OCaml effectively codes up lists as variants:


```ocaml
type 'a list = [] | :: of 'a * 'a list
```

* `[]` and `::` are constuctors. 
* Just a bit of syntactic magic to use `[]` and `::` as constructors rather than alphanumeric identifiers.

## OCaml Lists

In [ ]:
[]

In [ ]:
1::2::[]

can also write is as

In [ ]:
[1;2]

## `::` is right-associative

In [ ]:
1::2::[]

is equivalent to

In [ ]:
1::(2::[])

## Binary Trees

In [ ]:
type 'a tree = 
  Leaf 
| Node of 'a tree (* left *) * 'a (* value *) * 'a tree (* right *)

In OCaml, `(* ... *)` is how you write down comments in OCaml

In [ ]:
let t = Node (Leaf, 0.0, Node (Leaf, 1.0, Leaf))

## Null

"I call it my billion-dollar mistake. It was the invention
of the null reference in 1965. At that time, I was
**designing the first comprehensive type system for
references in an object-oriented language**. My goal was
to ensure that all use of references should be
**absolutely safe**, with checking performed automatically
by the compiler. **But I couldn’t resist the temptation to
put in a null reference, simply because it was so easy to
implement.** This has led to innumerable errors,
vulnerabilities, and system crashes, which have
probably caused a **billion dollars of pain and damage** in
the last forty years."

<h2 style="text-align:right"> - Sir Tony Hoare </h2>

## Option: A Built-in Variant

* OCaml does not have a null value. 

```ocaml
type 'a option = None | Some of 'a
```

<center>
    
<img src="images/box.jpg" width="150">
</center>

In [ ]:
None

In [ ]:
Some 10

In [ ]:
Some "Hello"

## When to use option types

```ocaml
type student = { name : string; rollno : string; 
                 marks : int}
```

* what value will you assign for `marks` field is the student is absent for an exam?
  + `0` is not a good answer since it might also be the case that the student actually scored 0.

```ocaml
type student = { name : string; rollno : string; 
                 marks : int option }
```

* Use `None` to indicate the exam has not been taken.

## Question

Given records, variants and tuples, which one would you pick for the following cases?

1. Represent currency denominations 10, 20, 50, 100, 200, 500, 2000.
2. Students who have name and roll numbers.
3. A dessert which has a sauce, a creamy component, and a crunchy component.

## Question

Given records, variants and tuples, which one would you pick for the following cases?

1. Represent currency denominations 10, 20, 50, 100, 200, 500, 2000.
```ocaml
type denomination = D10 | D20 | D50 | D100 | D200 | D500 | D2000
```
2. Students who have name and roll numbers.
```ocaml
type student = {name : string; roll_no : string}
```
3. A dessert which has a sauce, a creamy component, and a crunchy component.
```ocaml
type dessert = {sauce : string; creamy: string; crunchy: string}
```

## What about tuples?

Could have used tuples for 

2. Students who have name and roll numbers.
3. A dessert which has a sauce, a creamy component, and a crunchy component.

but

* Tuples are convenient for local uses
  + Returning a pair of values
* Records useful to global uses
  + _Tuples with documentation_

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>